In [1]:
%load_ext autoreload
%autoreload 2

import sys
# Add Python Tools
pythonToolsDir = r'J:\Projects\JHUTrabeculae\Software\Python\PythonTools'
sys.path.append(pythonToolsDir)

# Add ctk-cli
ctkcli_DIR = r'J:\Projects\JHUTrabeculae\Software\Python\ctk-cli'
sys.path.append(ctkcli_DIR)

# Add TrabeculaeTools
TrabeculaeTools_DIR = r'J:\Projects\JHUTrabeculae\Software\Python\TrabeculaeTools'
sys.path.append(TrabeculaeTools_DIR)

from ImageJTools import macroImageJ
from ImageJTools.HelperBoneJ import joinBoneJResults, extractSquareRoIs, maskImage, randomTransformation, getSimilarityMetrics

from PythonTools import registration, transforms, transformations
from PythonTools.helpers import vtk as vtkhelper
from PythonTools.helpers.elastix import elastix as elastixhelper
from PythonTools import io


import vtk
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import math
import os
import time
import seaborn as sns


import seaborn as sns
import pandas as pd

import SimpleITK as sitk

In [2]:
########################################################################################
##                Change the parameters here for a new analysis                       ##
########################################################################################

NRandomRoIs = 1 # Number of Random Generated RoIs
sizeRoImmVector = 3# list(np.arange(3,5.5,0.5))# [5,6,7,3] # RoI size in mm that will be tested
RandomSeed = 25 # Random Seed for repetitivity
NSegmentationAlgorithms = 1 # In a future, this could let use more than one algorithm


originalsPath = r'J:\Projects\JHUTrabeculae\Results\uCTvsCBCT\Originals'

# Evaluated Image path
uCTimagePath = originalsPath + r'\uCT.nii'
# Evaluated Image path
CBCTimagePath = originalsPath + r'\CBCT.nii'
# Total Bone mask for the evaluated Image path
uCTboneMaskPath = originalsPath + r'\uCT_BoneMask.nii'
# Total Bone mask for the evaluated Image path
CBCTboneMaskPath = originalsPath + r'\CBCT_BoneMask.nii'

# Path to save results of analysis
pathToSaveDirResults = r'J:\Projects\JHUTrabeculae\Results\uCTvsCBCT'

# Path to imageJ (Need BoneJ and NII reader plugins)
imagej_exe_path = r'J:\Projects\JHUTrabeculae\Software\Programs\Fiji\ImageJ-win64.exe'

# Macro definition for segmentation Trabeculae
fileXMLdescription = r'J:\Projects\JHUTrabeculae\Software\Python\TrabeculaeTools\ImageJMacros\SegmentTrabeculaImageJMacro.xml'
macroPath = r'J:\Projects\JHUTrabeculae\Software\Python\TrabeculaeTools\ImageJMacros\SegmentTrabeculaImageJMacro.ijm'
segmentTrabecula = macroImageJ(imagejPath = imagej_exe_path, macroPath = macroPath, xmlDefinition = fileXMLdescription)

# Macro definition for measure the Metrics
fileXMLdescription = r'J:\Projects\JHUTrabeculae\Software\Python\TrabeculaeTools\ImageJMacros\BoneJMetricsImageJMacro.xml'
macroPath = r'J:\Projects\JHUTrabeculae\Software\Python\TrabeculaeTools\ImageJMacros\BoneJMetricsImageJMacro.ijm'
BoneJMetrics = macroImageJ(imagejPath = imagej_exe_path, macroPath = macroPath, xmlDefinition = fileXMLdescription)

# Elastix registration
elastixPath = r'J:\Projects\JHUTrabeculae\Software\Programs\elastix\elastix.exe'
registrationParametersPath = originalsPath + r'\Registrationparams.txt'

# Donwsampling for registration
downFactor = 3
uCTimagePathDownsample = uCTimagePath[:-4] + '_resampled{0:.1f}.nii'.format(downFactor)
uCTimagePathDownsampleforTest = uCTimagePath[:-4] + '_resampled{0:.1f}.nii'.format(downFactor*4)

# List of Algorithm Used for segmentation
AlgorithmNameList = ['BoneJ']

In [3]:
# Image uCT is too big for registration
# We perform a resample of the original to downsample it

# Read uCT total Bone image
vtkReader = vtk.vtkNIFTIImageReader()
vtkReader.SetFileName(uCTimagePath)
vtkReader.Update()
uCTvoxelSize = np.array(vtkReader.GetOutput().GetSpacing());

# Reslice mask to match CBCT spacing
NewSpacing = downFactor * uCTvoxelSize

vtkReslicer = vtk.vtkImageReslice()
vtkReslicer.SetInputConnection( vtkReader.GetOutputPort() )
vtkReslicer.SetOutputSpacing( NewSpacing[0], NewSpacing[1], NewSpacing[2])
vtkReslicer.SetInterpolationModeToCubic()
vtkReslicer.SetOutputOrigin(0.0,0.0,0.0)
vtkReslicer.Update()


# Save the resulting image for registration
vtkWriter = vtk.vtkNIFTIImageWriter()
vtkWriter.SetFileName(uCTimagePathDownsample)
vtkWriter.SetInputConnection(vtkReslicer.GetOutputPort())
vtkWriter.Write()
vtkWriter.Update()

In [3]:
# Perform the initial registration for image flips
InitialTransform = originalsPath + r'\InitialTransform.tfm'
InitialTransformElastix = originalsPath + r'\InitialTransform.txt'
elastixhelper.itk2elastix(InitialTransform,InitialTransformElastix)

# This was made using Slicer and add to the header of the image.
#uCTimagePathDownsamplePreRegistered = uCTimagePath[:-4] + '_resampled{0:.1f}_initial.nii'.format(downFactor)

# Launch Registration


cmdString = elastixPath + \
            ' -f ' + CBCTimagePath + \
            ' -m ' + uCTimagePathDownsample + \
            ' -p ' + registrationParametersPath + \
            ' -t0 ' + InitialTransformElastix + \
            ' -out '+ originalsPath + \
            ' -threads 4'
            
print cmdString


J:\Projects\JHUTrabeculae\Software\Programs\elastix\elastix.exe -f J:\Projects\JHUTrabeculae\Results\uCTvsCBCT\Originals\CBCT.nii -m J:\Projects\JHUTrabeculae\Results\uCTvsCBCT\Originals\uCT_resampled3.0.nii -p J:\Projects\JHUTrabeculae\Results\uCTvsCBCT\Originals\Registrationparams.txt -t0 J:\Projects\JHUTrabeculae\Results\uCTvsCBCT\Originals\InitialTransform.txt -out J:\Projects\JHUTrabeculae\Results\uCTvsCBCT\Originals -threads 4


In [4]:
# Transform transformation file to itk 

RegistrationTransformElastix = originalsPath + r'\TransformParameters.0.txt'
RegistrationTransform = originalsPath + r'\TransformParameters.tfm'
elastixhelper.elastix2itk(RegistrationTransformElastix,RegistrationTransform)

# Compose Initial and Registration transformation
RegistrationTransformFinal = originalsPath + r'\RegistrationTransformFinal.tfm'

Tinitial = io.load_tfm(InitialTransform)
TReg = io.load_tfm(RegistrationTransform)

print 'Tinitial'
print '--------'
print Tinitial
print '____________________________________________________________________________'

print 'TReg'
print '----'
print TReg
print '____________________________________________________________________________'

TRegistrationComplete = transformations.concatenate_matrices(TReg, Tinitial)

print 'TRegistrationComplete'
print '---------------------'
print TRegistrationComplete
print '____________________________________________________________________________'
io.save_tfm(RegistrationTransformFinal,TRegistrationComplete)


Tinitial
--------
[[  3.74606593e-01   9.27183855e-01  -3.46944695e-18  -8.67403227e-01]
 [ -9.27042640e-01   3.74549539e-01  -1.74524064e-02   2.12170093e+01]
 [ -1.61815895e-02   6.53778652e-03   9.99847695e-01   3.70344274e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
____________________________________________________________________________
TReg
----
[[ 0.97580615  0.15702216  0.15213944 -3.17320495]
 [-0.14903707  0.9868474  -0.06261113  2.16880957]
 [-0.15996974  0.03842191  0.98637388  1.20884986]
 [ 0.          0.          0.          1.        ]]
____________________________________________________________________________
TRegistrationComplete
---------------------
[[  0.21751532   0.96455894   0.14937585  -0.63173776]
 [ -0.96966674   0.23102914  -0.07982446  23.21284763]
 [ -0.11150557  -0.12748175   0.9855531    2.52810405]
 [  0.           0.           0.           1.        ]]
_______________________________________________________________

In [5]:
# Generate Random RoIs for CBCT
# This control time information of the complete process

######################### This will change into for loop
sizeRoImm = 3

pathToSaveDir = pathToSaveDirResults
CBCTboneMaskPath = CBCTboneMaskPath
nRandomRoIs = 2

##########################

# Evaluated Image path
# uCTimagePath
# Evaluated Image path
# CBCTimagePath
# Total Bone mask for the evaluated Image path
# CBCTboneMaskPath

# Read Mask from CBCT
vtkReader = vtk.vtkNIFTIImageReader()
vtkReader.SetFileName(CBCTboneMaskPath)
vtkReader.Update()
# Get voxel Size from the image
voxelSize = np.array(vtkReader.GetOutput().GetSpacing());
SizeImage = np.array(vtkReader.GetOutput().GetExtent())[[1,3,5]];

# Calculate RoI size in voxels
roiSizeVoxels = np.round(sizeRoImm / voxelSize)
# Force to odd number of voxels
for i in range(3):
    if(roiSizeVoxels[i]/2 == 0):
        roiSizeVoxels[i] += 1
        
# Downsample CBCT mask to maximum voxel size for further eroding
maxSpacing =  roiSizeVoxels * np.array([voxelSize[0], voxelSize[1], voxelSize[2]])
vtkReslicer = vtk.vtkImageReslice()
vtkReslicer.SetInputConnection( vtkReader.GetOutputPort() )
vtkReslicer.SetOutputSpacing( maxSpacing[0], maxSpacing[1], maxSpacing[2])
vtkReslicer.SetOutputOrigin(0.0, 0.0, 0.0)
vtkReslicer.SetInterpolationModeToNearestNeighbor()
vtkReslicer.Update()

# Erode image 
maskDownsampled = vtkhelper.ImageData_to_array(vtkReslicer.GetOutput())
# Use 1x1x1 voxel mask
maskDownsampled = (ndimage.morphology.binary_erosion(maskDownsampled)).astype(int)

# Upsample CBCT eroded mask to original parameters
vtkReslicer = vtk.vtkImageReslice()
vtkReslicer.SetInputData( vtkhelper.ImageData_from_array(maskDownsampled))
vtkReslicer.SetOutputExtent(vtkReader.GetOutput().GetExtent())
vtkReslicer.SetOutputSpacing( voxelSize[0], voxelSize[1], voxelSize[2])
vtkReslicer.SetInterpolationModeToNearestNeighbor()
vtkReslicer.Update()

# Save Eroded mask

# Folder for RoI results
#saveDir = pathToSaveDir + '\\RoIMasks'
#if not os.path.exists(saveDir):
#    os.makedirs(saveDir)

#erodedImagePath = saveDir + '\\' + CBCTboneMaskPath.split("\\")[-1][:-4] + '_RoIMask' + str(sizeRoImm) + 'mm.nii'
#vtkWriter = vtk.vtkNIFTIImageWriter()
#vtkWriter.SetFileName(erodedImagePath)
#vtkWriter.SetInputConnection(vtkReslicer.GetOutputPort())
#vtkWriter.Write()
#vtkWriter.Update()

# Select random from mask for RoI file Creation
validVoxels =  vtkhelper.ImageData_to_array(vtkReslicer.GetOutput())
# Extract the idex for the centroids of masks
(Vx, Vy, Vz) = np.nonzero(validVoxels)
nFoundRoIs = len(Vx)

# Select Randoms
if( nFoundRoIs > 0):
    if( nRandomRoIs > 0 and nRandomRoIs <= nFoundRoIs):
        maxInt = sys.maxint - 1
        # Avoiding Overflow
        if (nFoundRoIs > maxInt):
            greaterTimes = nFoundRoIs/maxInt
            randomSample = np.random.random_integers(0, maxInt, nRandomRoIs)
            for i in range(greaterTimes):
                randomSample = randomSample.astype(long) + np.random.random_integers(0, maxInt, nRandomRoIs).astype(long)
        else:
            randomSample = np.random.random_integers(0, nFoundRoIs, nRandomRoIs)

        Vx = Vx[randomSample];
        Vy = Vy[randomSample];
        Vz = Vz[randomSample];

        nRoIs = nRandomRoIs
    else:
        nRoIs = nFoundRoIs

else:
    print "No RoIs for this size"

       
# Transform to world coordinates (mm)
Vx = Vx * voxelSize[0]
Vy = Vy * voxelSize[1]
Vz = Vz * voxelSize[2]
        
# Creating RoI file structure

RoIfileStructure = np.zeros((nRoIs,6))

for i in range(nRoIs):
    # Last element is the shape, 0: 'cube shape'
    RoIfileStructure[i,:] = np.array([i, Vx[i], Vy[i], Vz[i], sizeRoImm, 0 ])

# Writing the RoI file
saveDir = pathToSaveDir + '\\RandomRoIfiles'
if not os.path.exists(saveDir):
    os.makedirs(saveDir)

fileName = saveDir + '\\' + 'RandomRoIs' + str(sizeRoImm) + 'mm.cvs'
RoIfileStructureHeader = 'RoI.Number, Center.x.mm, Center.y.mm, Center.z.mm, RoI.Size.mm, RoI.Shape'
np.savetxt(fileName, \
           RoIfileStructure, \
           fmt='%10.5f', \
           delimiter=',', \
           newline='\n', \
           header=RoIfileStructureHeader, \
           comments='')


In [6]:
print RoIfileStructure

######################### This will change into for loop


RoIfileStructure = RoIfileStructure
pathToSaveDir = pathToSaveDirResults
uCTimagePath = uCTimagePath
CBCTimagePath = CBCTimagePath
RegistrationuCT2CBCTFile = RegistrationTransformFinal
CBCTOutput = 'CBCTOutput.nii'
uCTOutput = 'uCTOutput.nii'

##########################

# This is for for loop
RoIparameters = RoIfileStructure[0,:]
CurrentRoINumber = 0




## Extract CBCT RoI

# Read image
vtkReader = vtk.vtkNIFTIImageReader()
vtkReader.SetFileName(CBCTimagePath)
vtkReader.Update()
CBCTvoxelSize = np.array(vtkReader.GetOutput().GetSpacing());

# Calculating RoI
sizeRoImm = RoIparameters[4]
CBCTlimits = np.zeros((6))
CBCTlimits[0] = np.round( ( RoIparameters[1] - sizeRoImm/2.0 ) / CBCTvoxelSize[0] )
CBCTlimits[1] = np.round( ( RoIparameters[1] + sizeRoImm/2.0 ) / CBCTvoxelSize[0] )
CBCTlimits[2] = np.round( ( RoIparameters[2] - sizeRoImm/2.0 ) / CBCTvoxelSize[1] )
CBCTlimits[3] = np.round( ( RoIparameters[2] + sizeRoImm/2.0 ) / CBCTvoxelSize[1] )
CBCTlimits[4] = np.round( ( RoIparameters[3] - sizeRoImm/2.0 ) / CBCTvoxelSize[2] )
CBCTlimits[5] = np.round( ( RoIparameters[3] + sizeRoImm/2.0 ) / CBCTvoxelSize[2] )

# Get image data and crop RoI
image = vtkhelper.ImageData_to_array(vtkReader.GetOutput())
roi = image[CBCTlimits[0]:CBCTlimits[1], CBCTlimits[2]:CBCTlimits[3],CBCTlimits[4]:CBCTlimits[5]]
vtkroi = vtkhelper.ImageData_from_array(roi)
vtkroi.SetSpacing(CBCTvoxelSize[0], CBCTvoxelSize[1], CBCTvoxelSize[2])
vtkroi.SetOrigin(0,0,0)


# Writing the RoI file
saveDir = pathToSaveDir + '\\CBCTRoIs'
if not os.path.exists(saveDir):
    os.makedirs(saveDir)
roiImageFilepath = saveDir + '\\' + CBCTOutput
vtkWriter = vtk.vtkNIFTIImageWriter()
vtkWriter.SetFileName(roiImageFilepath)
vtkWriter.SetInputDataObject(vtkroi)
vtkWriter.Write()
vtkWriter.Update()


# Transformation of origin

T = np.eye(4)
T[3,0] = RoIparameters[1] - sizeRoImm/2.0
T[3,1] = RoIparameters[2] - sizeRoImm/2.0
T[3,2] = RoIparameters[3] - sizeRoImm/2.0

T = sitk.AffineTransform(3)
T.SetParameters([1,0,0,0,\
                 0,1,0,0,\
                 0,0,1,0,\
                 0,0,0,1])
image = sitk.Cast(sitk.ReadImage(roiImageFilepath), sitk.sitkUInt32)
sp = image.GetSpacing()
itkResampler = sitk.ResampleImageFilter() 
itkResampler.SetDefaultPixelValue(0) 
itkResampler.SetOutputSpacing(sp) 
itkResampler.SetOutputOrigin([RoIparameters[1] - sizeRoImm/2.0,\
                              RoIparameters[2] - sizeRoImm/2.0,\
                              RoIparameters[3] - sizeRoImm/2.0]) 
itkResampler.SetTransform(T) 
itkResampler.SetReferenceImage(image)
itkResampler.SetInterpolator(sitk.sitkBSpline)
image = itkResampler.Execute(image) 

sitk.WriteImage(image, roiImageFilepath)



## Extract uCT RoI

# Transform centroid of mask to Moving space
TReg = io.load_tfm(RegistrationuCT2CBCTFile)
InvTReg = np.linalg.inv(TReg)

Centroid = np.array([RoIparameters[1],RoIparameters[2],RoIparameters[3], 1.0])
TransformedCentroid = np.array([np.dot(InvTReg, Centroid)[:3]])

# Read image
vtkReader = vtk.vtkNIFTIImageReader()
vtkReader.SetFileName(CBCTimagePath)
vtkReader.Update()
uCTvoxelSize = np.array(vtkReader.GetOutput().GetSpacing());

# Calculating RoI
sizeRoImm = RoIparameters[4] * np.sqrt(2)
uCTlimits = np.zeros((6))
uCTlimits[0] = np.round( ( RoIparameters[1] - sizeRoImm/2.0 ) / uCTvoxelSize[0] )
uCTlimits[1] = np.round( ( RoIparameters[1] + sizeRoImm/2.0 ) / uCTvoxelSize[0] )
uCTlimits[2] = np.round( ( RoIparameters[2] - sizeRoImm/2.0 ) / uCTvoxelSize[1] )
uCTlimits[3] = np.round( ( RoIparameters[2] + sizeRoImm/2.0 ) / uCTvoxelSize[1] )
uCTlimits[4] = np.round( ( RoIparameters[3] - sizeRoImm/2.0 ) / uCTvoxelSize[2] )
uCTlimits[5] = np.round( ( RoIparameters[3] + sizeRoImm/2.0 ) / uCTvoxelSize[2] )

# Get image data and crop RoI
image = vtkhelper.ImageData_to_array(vtkReader.GetOutput())
roi = image[uCTlimits[0]:uCTlimits[1], uCTlimits[2]:uCTlimits[3],uCTlimits[4]:uCTlimits[5]]
vtkroi = vtkhelper.ImageData_from_array(roi)
vtkroi.SetSpacing(uCTvoxelSize[0], uCTvoxelSize[1], uCTvoxelSize[2])
vtkroi.SetOrigin(0,0,0)

# Writing the RoI file
saveDir = pathToSaveDir + '\\uCTRoIs'
if not os.path.exists(saveDir):
    os.makedirs(saveDir)
roiImageFilepath = saveDir + '\\' + uCTOutput
vtkWriter = vtk.vtkNIFTIImageWriter()
vtkWriter.SetFileName(roiImageFilepath)
vtkWriter.SetInputDataObject(vtkroi)
vtkWriter.Write()
vtkWriter.Update()

# Transform the RoI file back to CBCT space using registration
image = sitk.Cast(sitk.ReadImage(roiImageFilepath), sitk.sitkFloat32)
tx = sitk.ReadTransform(RegistrationuCT2CBCTFile)
image = sitk.Resample(image, tx, sitk.sitkLinear, sitk.sitkFloat32)
sitk.WriteImage(image, roiImageFilepath)


NameError: name 'RoIfileStructure' is not defined

In [6]:
######################### This will change into for loop


RoIfileStructure = RoIfileStructure
pathToSaveDir = pathToSaveDirResults
uCTimagePath = uCTimagePath
CBCTimagePath = CBCTimagePath
RegistrationuCT2CBCTFile = RegistrationTransformFinal
CBCTOutput = 'CBCTOutput.nii'
uCTOutput = 'uCTOutput.nii'

##########################

# This is for for loop
RoIparameters = RoIfileStructure[0,:]
CurrentRoINumber = 0
RoIparameters[1:4] = np.array([12.6,8.0,8.3])




## Extract CBCT RoI

imageCBCT = sitk.Cast(sitk.ReadImage(CBCTimagePath), sitk.sitkFloat32)
CBCTMatrixSize, CBCTvoxelSize = imageCBCT.GetSize(), imageCBCT.GetSpacing()

# Calculating RoI
sizeRoImm = RoIparameters[4]

startFilterRoI = list(np.round( (RoIparameters[1:4] - sizeRoImm/2.0) / CBCTvoxelSize).astype(int))
sizeFilterRoI = list(np.round((sizeRoImm )/ CBCTvoxelSize ).astype(int))

print startFilterRoI
print sizeFilterRoI

itkExtractRoI = sitk.RegionOfInterestImageFilter()
itkExtractRoI.SetIndex(startFilterRoI)
itkExtractRoI.SetSize(sizeFilterRoI)
imageRoi = itkExtractRoI.Execute(imageCBCT)

# Writing the RoI file
saveDir = pathToSaveDir + '\\CBCTRoIs'
if not os.path.exists(saveDir):
    os.makedirs(saveDir)
roiImageFilepathCBCT = saveDir + '\\' + CBCTOutput

sitk.WriteImage(imageRoi, roiImageFilepathCBCT)


## Extract uCT RoI

image = sitk.Cast(sitk.ReadImage(uCTimagePath), sitk.sitkFloat32)
uCTMatrixSize, uCTvoxelSize = image.GetSize(), image.GetSpacing()


# Calculating RoI
sizeRoImm = RoIparameters[4] * np.sqrt(2.0)

# Transform start point of mask to Moving space
TReg = io.load_tfm(RegistrationuCT2CBCTFile)
InvTReg = np.linalg.inv(TReg)

centroid = np.array([RoIparameters[1],RoIparameters[2],RoIparameters[3], 1.0])
centroidTransformed = np.dot(TReg, centroid)[:3]

startFilterRoI = list((np.round( (centroidTransformed - sizeRoImm/2.0) / uCTvoxelSize)).astype(int))

sizeFilterRoI = list(np.round((sizeRoImm )/ uCTvoxelSize ).astype(int))


print startFilterRoI
print sizeFilterRoI

itkExtractRoI = sitk.RegionOfInterestImageFilter()
itkExtractRoI.SetIndex(startFilterRoI)
itkExtractRoI.SetSize(sizeFilterRoI)
imageRoi = itkExtractRoI.Execute(image)
image = None

# Writing the RoI file
saveDir = pathToSaveDir + '\\uCTRoIs'
if not os.path.exists(saveDir):
    os.makedirs(saveDir)
roiImageFilepath = saveDir + '\\' + uCTOutput

sitk.WriteImage(imageRoi, roiImageFilepath)

# Register RoI
imageRoI = sitk.Cast(sitk.ReadImage(roiImageFilepath), sitk.sitkFloat32)
imageRoICBCT = sitk.Cast(sitk.ReadImage(roiImageFilepathCBCT), sitk.sitkFloat32)

# Resample RoI CBCT to uCT Spacing
itkResampler = sitk.ResampleImageFilter()
itkResampler.SetInterpolator(sitk.sitkNearestNeighbor)
itkResampler.SetOutputSpacing(imageRoI.GetSpacing())
itkResampler.SetDefaultPixelValue(0)
itkResampler.SetTransform(sitk.Transform())
itkResampler.SetOutputOrigin(imageRoICBCT.GetOrigin())
newSize = - 2 + np.floor(np.array(imageRoICBCT.GetSize()) *\
                         np.array(imageRoICBCT.GetSpacing()) /\
                         np.array(imageRoI.GetSpacing())).astype(int)
itkResampler.SetSize(list(newSize))
imageRoICBCT = itkResampler.Execute(imageRoICBCT)


# Transform RoI uCT to CBCT geometrical Space
T = sitk.ReadTransform(RegistrationuCT2CBCTFile)

imageRoI = sitk.Resample(imageRoI, imageRoICBCT, T, sitk.sitkBSpline, sitk.sitkFloat32)
sitk.WriteImage(imageRoI, roiImageFilepath)

imageRoI = None
imageRoICBCT = None



[148, 87, 91]
[40, 40, 40]
[639, 719, 440]
[303, 303, 303]


In [24]:
## Test for eroding mask

sizeRoImm = RoIparameters[4]
CBCTboneMask = sitk.Cast(sitk.ReadImage(CBCTboneMaskPath), sitk.sitkUInt8)

# Calculate Radius
radius = np.round(sizeRoImm / np.array(CBCTboneMask.GetSpacing())).astype(int)

# Create filter
itkEroder = sitk.BinaryErodeImageFilter()
itkEroder.SetKernelType(sitk.sitkBox)
itkEroder.SetKernelRadius(radius)
itkEroder.SetForegroundValue(255)
itkEroder.SetBackgroundValue(0.0)
CBCTboneMaskEroded = itkEroder.Execute(CBCTboneMask)

saveFile = r'J:\Projects\JHUTrabeculae\Results\uCTvsCBCT\RandomRoIfiles\CBCTErodedMask.nii'
sitk.WriteImage(CBCTboneMaskEroded, saveFile)




3.0

[   0.          106.          129.          189.            4.            0.
    7.95000032    9.67500038    2.4750001 ]
#########################################################
               Generating RoI from Image
#########################################################
[0.00 s]    - RoI parameters:
[0.00 s]        - RoI #              0.0
[0.00 s]        - RoI Center voxel   [ 106.  129.  189.]  voxel
[0.00 s]        - RoI Size (mm)      4.0
[0.00 s]        - RoI Center (mm)      [ 7.95000032  9.67500038  2.4750001 ]
        - RoI Shape          Cuboid
[[  0.21751532  -0.96966674  -0.11150557  22.92803669]
 [  0.96455894   0.23102914  -0.12748175  -4.43120871]
 [  0.14937585  -0.07982446   0.9855531   -0.54426149]
 [  0.           0.           0.           1.        ]]
[0.00 s]        - RoI Center Transformed (mm)      [[ 14.99978115   5.15672477   2.31021893]]
